# 1. EDA (Exploratory Data Analysis)

Bu notebook veri setinin ilk incelemesi, degiskenlerin analizi, dagilimlarin ve korelasyonlarin incelenmesini icerir.

## Icerik
1. Veri Yukleme
2. Temel Istatistikler
3. Target Analizi
4. Eksik Deger Analizi
5. Kategorik Degisken Analizi
6. Numerik Degisken Analizi
7. Korelasyon Analizi
8. Bulgular ve Sonuc

In [ ]:
# Kutuphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Gorsellestirme ayarlari
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Pandas ayarlari
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## 1. Veri Yukleme

In [ ]:
# Veri yolu
DATA_PATH = Path('../data/raw/')

# Veriyi yukle
# df = pd.read_csv(DATA_PATH / 'train.csv')
# print(f'Veri yuklendi: {df.shape[0]} satir, {df.shape[1]} sutun')

## 2. Temel Istatistikler

In [ ]:
# Numerik degiskenler icin istatistikler
# df.describe()

## 3. Target Analizi

In [ ]:
# Target dagilimi
# TARGET = 'target'  # Kendi target sutununuzu yazin
# print(df[TARGET].value_counts())

## 4. Eksik Deger Analizi

In [ ]:
# Eksik deger sayilari
# missing = df.isnull().sum()
# missing[missing > 0]

## 5. Kategorik Degisken Analizi

In [ ]:
# Kategorik sutunlar
# cat_cols = df.select_dtypes(include=['object']).columns.tolist()
# print(f'Kategorik sutun sayisi: {len(cat_cols)}')

## 6. Numerik Degisken Analizi

In [ ]:
# Numerik sutunlar
# num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# print(f'Numerik sutun sayisi: {len(num_cols)}')

## 7. Korelasyon Analizi

In [ ]:
# Korelasyon matrisi
# plt.figure(figsize=(15, 12))
# sns.heatmap(df[num_cols].corr(), cmap='coolwarm', center=0)
# plt.title('Korelasyon Matrisi')
# plt.show()

## 8. Bulgular ve Sonuc

### Onemli Bulgular
1. [Bulgu 1]
2. [Bulgu 2]

### Sonraki Adimlar
1. [Adim 1]
2. [Adim 2]